<a href="https://colab.research.google.com/github/camilamaestrelli/Employee-Churn-Classification-Competition/blob/main/Pycaret_Algorithm_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install pycaret


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from  pycaret.classification import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
train_url = 'https://raw.githubusercontent.com/camilamaestrelli/Employee-Churn-Classification-Competition/main/treino.csv'
test_url = 'https://raw.githubusercontent.com/camilamaestrelli/Employee-Churn-Classification-Competition/main/teste.csv'

# importing the datasets
original_train = pd.read_csv(train_url, sep = ';', decimal = ',')
original_test = pd.read_csv(test_url, sep = ';', decimal = ',')

# copying the datsets
df_train = original_train.copy()
df_test = original_test.copy()

## Transformações

In [5]:
dict_turnover = {'SIM': 1, 
                 'NAO': 0}

df_train['turnover_apos_1_ano'] = df_train['turnover_apos_1_ano'].map(dict_turnover)


In [6]:
dict_sexo = {'FEMININO': 1, 
                 'MASCULINO': 0}

df_train['func_sexo'] = df_train['func_sexo'].map(dict_sexo)
df_test['func_sexo'] = df_test['func_sexo'].map(dict_sexo)

In [7]:
dict_racacor = {'SEM INFO': 0, 
                'PRETA-PARDA': 1, 
                'AMARELA-BRANCA':2, 
                'INDIGENA':3}

df_train['func_racacor'] = df_train['func_racacor'].map(dict_racacor)
df_test['func_racacor'] = df_test['func_racacor'].map(dict_racacor)

In [8]:
dict_escolar = {'ANALFABETO-FUND_INCOMPLETO': 1, 
                'FUND_COMPLETO-MEDIO_INCOMPLETO': 2, 
                'MEDIO COMPLETO':3, 
                'SUPERIOR_INCOMPLETO':4,
                'SUPERIOR_COMPLETO':5}

df_train['func_escolaridade'] = df_train['func_escolaridade'].map(dict_escolar)
df_test['func_escolaridade'] = df_test['func_escolaridade'].map(dict_escolar)


In [9]:
df_train.contrato_salario = df_train.contrato_salario/1000
df_test.contrato_salario = df_test.contrato_salario/1000

In [10]:
dict_uf = {'MA': 1,
           'AL': 2,
           'PI': 3,
           'CE': 4,
           'BA': 5,
           'RN': 6,
           'SE': 7,
           'PB': 8,
           'PE': 9
          }
           
df_train['func_uf'] = df_train['func_uf'].map(dict_uf)
df_test['func_uf'] = df_test['func_uf'].map(dict_uf)

In [11]:
dict_deficiencia = {'SIM': 1, 
                    'NAO': 0}

df_train['func_deficiencia'] = df_train['func_deficiencia'].map(dict_deficiencia)
df_test['func_deficiencia'] = df_test['func_deficiencia'].map(dict_deficiencia)

In [12]:
dict_porte = {'ATE 4': 1,
              'DE 5 A 9': 2,
              'DE 10 A 19': 3,
              'DE 20 A 49': 4,                   
              'DE 50 A 99': 5,
              'DE 100 A 249': 6,
              'DE 250 A 499': 7,
              'DE 500 A 999': 8,
              '1000 OU MAIS': 9
             }

df_train['empresa_porte'] = df_train['empresa_porte'].map(dict_porte)
df_test['empresa_porte'] = df_test['empresa_porte'].map(dict_porte)

In [13]:
dict_setor = {'Agricultura': 1,
              'Construçao civil': 2,
              'Comércio': 3,
              'Indústria': 4,
              'Serviços': 5,
              'Administraçao pública': 6
             }

df_train['empresa_setor'] = df_train['empresa_setor'].map(dict_setor)
df_test['empresa_setor'] = df_test['empresa_setor'].map(dict_setor)

In [14]:
def aux_age(x):
    if x <= 20:
        return 1
    elif x <= 25:
        return 2
    elif x <=30:
        return 3
    elif x <=35:
        return 4
    elif x<=40:
        return 5
    elif x<=45:
        return 6
    elif x <= 50:
        return 7
    elif x <= 55:
        return 8
    else:
        return 9
    
df_train['func_idade'] = df_train['func_idade'].map(aux_age)
df_test['func_idade'] = df_test['func_idade'].map(aux_age)

In [15]:
# function to reverse the dict: turn keys to values and values to keys 
def reverse_dict(dic):
    reverse_dic = dict()
    for key in dic:
        val = dic[key]
        reverse_dic[val] = key
    return reverse_dic

# another way to do it:
# inverse_dict = {v: k for k, v in dic.items()}


## Pycaret

In [16]:
class_setup = setup(data = df_train, target = 'turnover_apos_1_ano', preprocess = True, categorical_features = None, 
                    session_id = 1986,
                    #high_cardinality_features = ['contrato_horas_trabalho'], high_cardinality_method = 'frequency', 
                    #numeric_features = None,
                    normalize = True, normalize_method = 'zscore', transformation = False, 
                    transformation_method = 'yeo-johnson', handle_unknown_categorical = True,                                      
                    remove_multicollinearity = True, multicollinearity_threshold = 0.9, remove_perfect_collinearity = True, 
                    create_clusters = True, cluster_iter = 20,  
                    feature_selection = True, feature_selection_threshold = 0.8, 
                    feature_selection_method = 'classic', feature_interaction = True, 
                    feature_ratio = True, interaction_threshold = 0.01, fix_imbalance = True, 
                    fix_imbalance_method = None, data_split_shuffle = True, 
                    data_split_stratify = False, fold_strategy = 'stratifiedkfold', 
                    fold = 10, fold_shuffle = False, 
                    n_jobs = - 1, use_gpu = True, 
                    silent = True, verbose = True, 
                    
                   )

class_setup

,Description,Value
0,session_id,1986
1,Target,turnover_apos_1_ano
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(15000, 11)"
5,Missing Values,False
6,Numeric Features,2
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


({'ada': <pycaret.containers.models.classification.AdaBoostClassifierContainer at 0x7f08c62d1510>,
  'dt': <pycaret.containers.models.classification.DecisionTreeClassifierContainer at 0x7f08b6683690>,
  'dummy': <pycaret.containers.models.classification.DummyClassifierContainer at 0x7f08c64d1110>,
  'et': <pycaret.containers.models.classification.ExtraTreesClassifierContainer at 0x7f08c64d1210>,
  'gbc': <pycaret.containers.models.classification.GradientBoostingClassifierContainer at 0x7f08c642ce50>,
  'gpc': <pycaret.containers.models.classification.GaussianProcessClassifierContainer at 0x7f08b6828750>,
  'knn': <pycaret.containers.models.classification.KNeighborsClassifierContainer at 0x7f08b6683350>,
  'lda': <pycaret.containers.models.classification.LinearDiscriminantAnalysisContainer at 0x7f08b6828690>,
  'lightgbm': <pycaret.containers.models.classification.LGBMClassifierContainer at 0x7f08c64d15d0>,
  'lr': <pycaret.containers.models.classification.LogisticRegressionClassifierCo

In [17]:
top3 = compare_models(n_select = 3, turbo = True, sort = 'F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8307,0.9122,0.9056,0.7078,0.7944,0.6541,0.6687,8.651
lightgbm,Light Gradient Boosting Machine,0.8385,0.9168,0.8587,0.7378,0.7934,0.6621,0.6676,1.397
rf,Random Forest Classifier,0.8323,0.9094,0.8553,0.7285,0.7865,0.6499,0.6560,4.217
ada,Ada Boost Classifier,0.8231,0.9016,0.8890,0.7017,0.7842,0.6379,0.6509,2.295
et,Extra Trees Classifier,0.8312,0.9076,0.8160,0.7428,0.7774,0.6420,0.6441,3.678
knn,K Neighbors Classifier,0.8029,0.8709,0.8584,0.6804,0.7589,0.5960,0.6079,2.064
lr,Logistic Regression,0.8079,0.8724,0.7793,0.7152,0.7456,0.5918,0.5936,3.861
dt,Decision Tree Classifier,0.7848,0.7688,0.7108,0.6995,0.7046,0.5355,0.5360,0.917
ridge,Ridge Classifier,0.7405,0.0000,0.6773,0.6318,0.6534,0.4466,0.4476,0.533
lda,Linear Discriminant Analysis,0.7403,0.7957,0.6770,0.6314,0.6531,0.4460,0.4470,0.686


In [18]:
lgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8514,0.9173,0.8654,0.7575,0.8079,0.6876,0.6917
1,0.8438,0.9189,0.8839,0.7363,0.8034,0.6756,0.6834
2,0.8362,0.9080,0.8470,0.7379,0.7887,0.6560,0.6601
3,0.8400,0.9205,0.8945,0.7259,0.8014,0.6699,0.6801
4,0.8333,0.9171,0.8575,0.7287,0.7879,0.6521,0.6580
5,0.8276,0.9126,0.8364,0.7271,0.7779,0.6382,0.6424
6,0.8362,0.9210,0.8447,0.7396,0.7887,0.6559,0.6598
7,0.8314,0.9143,0.8842,0.7164,0.7915,0.6526,0.6628
8,0.8524,0.9220,0.8474,0.7685,0.8060,0.6873,0.6895


In [19]:
tuned_lgbm = tune_model(lgbm, optimize='f1', n_iter = 50)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8448,0.9191,0.9156,0.7259,0.8098,0.6816,0.6947
1,0.8229,0.9117,0.9050,0.6957,0.7867,0.6396,0.6558
2,0.8381,0.9075,0.9024,0.7200,0.8009,0.6674,0.6795
3,0.8333,0.9134,0.9261,0.7048,0.8005,0.6618,0.6801
4,0.8333,0.9175,0.9103,0.7099,0.7977,0.6596,0.6744
5,0.8219,0.9089,0.8918,0.6983,0.7833,0.6359,0.6496
6,0.8333,0.9188,0.9053,0.7122,0.7972,0.6591,0.6728
7,0.8200,0.9064,0.9395,0.6826,0.7907,0.6396,0.6648
8,0.8448,0.9188,0.8947,0.7343,0.8066,0.6791,0.6883


In [20]:
gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8457,0.9154,0.9182,0.7265,0.8112,0.6837,0.6971
1,0.8210,0.9131,0.9050,0.6929,0.7849,0.6361,0.6528
2,0.8314,0.9056,0.8971,0.7113,0.7935,0.6542,0.6668
3,0.8324,0.9118,0.9235,0.7042,0.7991,0.6597,0.6776
4,0.8400,0.9103,0.9235,0.7157,0.8065,0.6738,0.6897
5,0.8219,0.9113,0.8918,0.6983,0.7833,0.6359,0.6496
6,0.8257,0.9167,0.9000,0.7023,0.7889,0.6443,0.6587
7,0.8162,0.9061,0.9184,0.6830,0.7834,0.6297,0.6505
8,0.8448,0.9202,0.9079,0.7294,0.8089,0.6808,0.6924


In [ ]:
tuned_gbc = tune_model(gbc, optimize='f1', n_iter = 15)

In [22]:
rf = create_model('rf')

In [ ]:
tuned_rf = tune_model(rf, optimize='f1', n_iter = 20)

In [24]:
tuned_gbc1 = tune_model(gbc, optimize='f1', n_iter = 20)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8476,0.9205,0.8707,0.7483,0.8049,0.6810,0.6863
1,0.8400,0.9140,0.8892,0.7279,0.8005,0.6691,0.6785
2,0.8371,0.9063,0.8681,0.7311,0.7937,0.6608,0.6675
3,0.8352,0.9145,0.9024,0.7155,0.7981,0.6621,0.6748
4,0.8381,0.9092,0.8945,0.7228,0.7995,0.6663,0.6769
5,0.8362,0.9145,0.8734,0.7275,0.7938,0.6598,0.6673
6,0.8390,0.9165,0.8816,0.7298,0.7986,0.6665,0.6748
7,0.8295,0.9063,0.9132,0.7039,0.7950,0.6532,0.6694
8,0.8495,0.9208,0.8658,0.7546,0.8064,0.6843,0.6886


In [25]:
tuned_lgbm1 = tune_model(lgbm, optimize='f1', n_iter = 60)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8467,0.9183,0.8786,0.7433,0.8053,0.6803,0.6868
1,0.8362,0.9176,0.8945,0.7197,0.7976,0.6627,0.6738
2,0.8457,0.9081,0.8865,0.7385,0.8058,0.6795,0.6874
3,0.8343,0.9163,0.9024,0.7140,0.7972,0.6603,0.6732
4,0.8333,0.9154,0.8892,0.7170,0.7939,0.6567,0.6674
5,0.8305,0.9153,0.8786,0.7161,0.7891,0.6498,0.6593
6,0.8333,0.9207,0.8789,0.7214,0.7924,0.6554,0.6644
7,0.8219,0.9132,0.9026,0.6957,0.7858,0.6377,0.6536
8,0.8543,0.9225,0.8684,0.7621,0.8118,0.6937,0.6977


In [26]:
top = [tuned_gbc, tuned_lgbm, tuned_lgbm1]

In [27]:
combinacao_soft = blend_models(top, optimize = 'f1', method = 'soft', choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8467,0.9216,0.8760,0.7444,0.8048,0.6799,0.6861
1,0.8381,0.9176,0.8971,0.7219,0.8000,0.6667,0.6778
2,0.8400,0.9079,0.8786,0.7319,0.7986,0.6677,0.6754
3,0.8381,0.9172,0.9024,0.7200,0.8009,0.6674,0.6795
4,0.8333,0.9139,0.8892,0.7170,0.7939,0.6567,0.6674
5,0.8333,0.9167,0.8707,0.7237,0.7904,0.6540,0.6617
6,0.8295,0.9208,0.8763,0.7161,0.7882,0.6479,0.6572
7,0.8229,0.9118,0.9053,0.6964,0.7872,0.6398,0.6560
8,0.8543,0.9223,0.8711,0.7609,0.8123,0.6941,0.6983


In [28]:
combinacao_hard = blend_models(top, optimize = 'f1', method = 'hard', choose_better = True)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8448,0.0,0.8865,0.7368,0.8048,0.6777,0.6857
1,0.8352,0.0,0.9077,0.7137,0.7991,0.6628,0.6766
2,0.8429,0.0,0.8918,0.7316,0.8038,0.6749,0.6841
3,0.8362,0.0,0.9182,0.7117,0.8018,0.6660,0.6817
4,0.8343,0.0,0.8997,0.7149,0.7967,0.6599,0.6724
5,0.8286,0.0,0.8813,0.7122,0.7877,0.6467,0.6570
6,0.8295,0.0,0.8816,0.7143,0.7892,0.6487,0.6588
7,0.8286,0.0,0.9263,0.6984,0.7964,0.6533,0.6728
8,0.8514,0.0,0.8737,0.7545,0.8098,0.6889,0.6939


In [37]:
stacker = stack_models(top)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8419,0.9213,0.8786,0.7351,0.8005,0.6713,0.6786
1,0.8333,0.9139,0.8892,0.7170,0.7939,0.6567,0.6674
2,0.8410,0.9062,0.8734,0.7356,0.7986,0.6687,0.6755
3,0.8419,0.9192,0.9077,0.7242,0.8056,0.6752,0.6874
4,0.8352,0.9120,0.8918,0.7191,0.7962,0.6606,0.6714
5,0.8219,0.9138,0.8681,0.7060,0.7787,0.6323,0.6418
6,0.8295,0.9167,0.8737,0.7171,0.7877,0.6476,0.6564
7,0.8238,0.9086,0.9132,0.6954,0.7895,0.6427,0.6604
8,0.8581,0.9300,0.8816,0.7631,0.8181,0.7027,0.7077


In [29]:
pred_final = predict_model(combinacao_hard, data = df_test)
pred_final

,func_sexo,func_idade,func_racacor,func_escolaridade,func_uf,func_deficiencia,empresa_porte,empresa_setor,contrato_horastrabalho,contrato_salario,Label
0,1,3,1,3,1,0,1,3,44,1.58319,1
1,0,2,0,3,5,0,2,4,44,0.97995,0
2,1,3,0,3,6,0,1,3,44,1.01585,0
3,0,7,1,3,5,0,2,3,44,1.08945,0
4,1,4,0,3,5,0,1,5,44,1.28162,0
...,...,...,...,...,...,...,...,...,...,...,...
14995,1,3,2,3,3,0,5,5,44,1.14291,0
14996,1,2,1,3,5,0,3,5,44,1.38588,0
14997,1,5,0,5,9,0,4,5,40,5.85413,1
14998,0,6,0,3,5,0,3,5,22,0.50000,0


In [30]:
pred_final['Label'].value_counts()

0    8054
1    6946
Name: Label, dtype: int64

In [31]:
pred_final['Label'] 

0        1
1        0
2        0
3        0
4        0
        ..
14995    0
14996    0
14997    1
14998    0
14999    0
Name: Label, Length: 15000, dtype: int64

In [32]:
dict_turnover = { 1 : 'SIM', 
                  0 : 'NAO'}

pred_final['Label']  = pred_final['Label'].map(dict_turnover)

In [33]:
pred_final['Label'] 

0        SIM
1        NAO
2        NAO
3        NAO
4        NAO
        ... 
14995    NAO
14996    NAO
14997    SIM
14998    NAO
14999    NAO
Name: Label, Length: 15000, dtype: object

In [34]:
pred_final['Label'].value_counts()

NAO    8054
SIM    6946
Name: Label, dtype: int64

In [35]:
subm = pd.DataFrame()
subm['turnover_apos_1_ano'] = pred_final['Label'] 
subm.to_csv('submissão.csv', index = False)

In [36]:
!pip list

Package                       Version
----------------------------- ------------------------------
absl-py                       1.0.0
alabaster                     0.7.12
albumentations                0.1.12
alembic                       1.8.0
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.0
blis                          0.4.1
bokeh                         2.3.3
Boruta                        0.3
Bottleneck                    1.3.